In [1]:
import os
import pandas as pd

RUTA_PADRE = "../data"

RUTA_LIMPIO = os.path.join(RUTA_PADRE, "interim", "ecu911", "ecu911_limpio.csv")
RUTA_CATALOGO = os.path.join(RUTA_PADRE, "processed", "catalogo_parroquias_ecuador.csv")

SALIDA_PREPRO = os.path.join(RUTA_PADRE, "processed", "ecu911", "ecu911_con_coords.csv")
os.makedirs(os.path.dirname(SALIDA_PREPRO), exist_ok=True)

print("Ruta limpio:", RUTA_LIMPIO)
print("Ruta catálogo:", RUTA_CATALOGO)
print("Salida:", SALIDA_PREPRO)

Ruta limpio: ../data\interim\ecu911\ecu911_limpio.csv
Ruta catálogo: ../data\processed\catalogo_parroquias_ecuador.csv
Salida: ../data\processed\ecu911\ecu911_con_coords.csv


In [2]:
def normalizar_columnas(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.replace("ï»¿", "", regex=False)
        .str.strip()
        .str.lower()
    )
    return df

def estandarizar_cod_parroquia(serie: pd.Series) -> pd.Series:
    s = serie.astype(str).str.strip()
    s = s.str.replace(".0", "", regex=False)
    s = s.str.zfill(6)
    s = s.replace({"nan": pd.NA, "none": pd.NA, "": pd.NA})
    return s


In [3]:
try:
    print("Cargando dataset limpio...")
    df_911 = pd.read_csv(RUTA_LIMPIO, low_memory=False)
    df_911 = normalizar_columnas(df_911)
    print("Registros:", len(df_911))
    print("Columnas:", list(df_911.columns))
except Exception as e:
    raise RuntimeError(f"Error cargando dataset limpio: {e}")


Cargando dataset limpio...
Registros: 1773973
Columnas: ['provincia', 'canton', 'cod_parroquia', 'parroquia', 'servicio', 'subtipo', 'fecha']


In [4]:
try:
    print("Cargando catálogo de parroquias...")
    catalogo = pd.read_csv(RUTA_CATALOGO, dtype={"cod_parroquia": str}, low_memory=False)
    catalogo = normalizar_columnas(catalogo)
    print("Filas catálogo:", len(catalogo))
    print("Columnas catálogo:", list(catalogo.columns))
except Exception as e:
    raise RuntimeError(f"Error cargando catálogo: {e}")


Cargando catálogo de parroquias...
Filas catálogo: 1257
Columnas catálogo: ['cod_parroquia', 'provincia', 'canton', 'parroquia', 'direccion_busqueda', 'lat', 'lon']


In [5]:
# Validar que exista cod_parroquia en ambos
if "cod_parroquia" not in df_911.columns:
    raise ValueError("El dataset limpio no tiene 'cod_parroquia'. Revisa nombres de columnas.")

if "cod_parroquia" not in catalogo.columns:
    raise ValueError("El catálogo no tiene 'cod_parroquia'. Revisa nombres de columnas.")

# Estandarizar
df_911["cod_parroquia"] = estandarizar_cod_parroquia(df_911["cod_parroquia"])
catalogo["cod_parroquia"] = estandarizar_cod_parroquia(catalogo["cod_parroquia"])

# Validar lat/lon
# Si en tu catálogo se llaman distinto, cámbialos aquí.
lat_col = "lat"
lon_col = "lon"

if lat_col not in catalogo.columns or lon_col not in catalogo.columns:
    raise ValueError(f"El catálogo no tiene columnas '{lat_col}'/'{lon_col}'. Columnas disponibles: {list(catalogo.columns)}")

# Mantener solo lo necesario del catálogo para evitar columnas repetidas
catalogo_coords = catalogo[["cod_parroquia", lat_col, lon_col]].drop_duplicates(subset=["cod_parroquia"])
print("Catálogo coords (únicos por cod_parroquia):", len(catalogo_coords))


Catálogo coords (únicos por cod_parroquia): 1048


In [6]:
antes = len(df_911)

df_merge = df_911.merge(
    catalogo_coords,
    on="cod_parroquia",
    how="left"
)

print("Registros tras merge:", len(df_merge), "(debería ser igual a antes:", antes, ")")


Registros tras merge: 1773973 (debería ser igual a antes: 1773973 )


In [7]:
total = len(df_merge)
sin_latlon = df_merge[lat_col].isna().sum()

print("Total registros:", total)
print("Sin coordenadas:", sin_latlon)
print("Con coordenadas:", total - sin_latlon)
print("Match rate:", f"{(total - sin_latlon)/total*100:.2f}%")

# Top parroquias sin match (para depuración)
top_sin_match = (
    df_merge[df_merge[lat_col].isna()]
    .groupby("cod_parroquia")
    .size()
    .sort_values(ascending=False)
    .head(15)
)

print("\nTop cod_parroquia sin match (conteo de llamadas):")
print(top_sin_match)


Total registros: 1773973
Sin coordenadas: 638
Con coordenadas: 1773335
Match rate: 99.96%

Top cod_parroquia sin match (conteo de llamadas):
cod_parroquia
900451    245
900151    195
140157    140
220254     31
011551     27
dtype: int64


In [8]:
df_merge["fecha"].dtype

dtype('O')

In [9]:
df_merge = df_merge.sort_values(by="fecha", ascending=True)
df_merge = df_merge.reset_index(drop=True)


In [10]:
# Opcional: quedarte solo con filas con coordenadas
df_final = df_merge.dropna(subset=[lat_col, lon_col]).copy()

df_final.to_csv(SALIDA_PREPRO, index=False, encoding="utf-8")
print("Guardado:", SALIDA_PREPRO)
print("Registros finales con coords:", len(df_final))


Guardado: ../data\processed\ecu911\ecu911_con_coords.csv
Registros finales con coords: 1773335


In [11]:
df_final["fecha"].head()

0    2025-01-01
1    2025-01-01
2    2025-01-01
3    2025-01-01
4    2025-01-01
Name: fecha, dtype: object

In [12]:
df_final["fecha"].tail()

1773968    2025-10-31
1773969    2025-10-31
1773970    2025-10-31
1773971    2025-10-31
1773972    2025-10-31
Name: fecha, dtype: object